In [57]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Run VAE models systematically

## Imports

In [58]:
import pandas as pd
import itertools
import os
from evoscaper.scripts.cvae_scan import main as cvae_scan
from evoscaper.utils.preprocess import make_datetime_str

## Create table of all VAE model training settings

Parameters for:
- Biological dataset generation
- Training data
    - Input
    - Output 
- Model architecture
- Training hyperparameters

### Initial parameters

In [59]:
data_dir = '../data'

hpos_architecture = {
    'seed_arch': 1,
    'hidden_size': 16,
    'enc_ls': 32,
    'dec_ls': 32,
    'num_enc_layers': 2,
    'num_dec_layers': 2,
    'factor_expanding_ls': 1,
    'factor_contracting_ls': 1,
    'model': 'CVAE',
    'use_sigmoid_decoder': False,
    'enc_init': 'HeNormal',
    'dec_init': 'HeNormal',
    'init_model_with_random': True,
    'activation': 'leaky_relu',
}


hpos_training = {
    'seed_train': 1,
    'batch_size': 128,
    'epochs': 2000,
    'patience': 500,
    'learning_rate': 1e-1,
    'loss_func': 'mse',
    'use_dropout': False,
    'dropout_rate': 0.1,
    'use_l2_reg': False,
    'l2_reg_alpha': 0.01,
    'use_kl_div': True,
    'kl_weight': 2.5e-4,  # inspired by https://github.com/elttaes/VAE-MNIST-Haiku-Jax/blob/main/cVAE_mnist.ipynb
}
hpos_training['print_every'] = hpos_training['epochs'] // 100

hpos_optimization = {
    'seed_opt': 1,
    'opt_method': 'adam',
    'opt_min_lr': 1e-6,
    'opt_min_delta': 1e-4,
    'learning_rate_sched': 'cosine_decay',
    'use_warmup': True,
    'warmup_epochs': 20,
}

hpos_dataset = {
    'seed_dataset': 1,
    'include_diffs': False,
    'objective_col': 'Log sensitivity',
    'output_species': ('RNA_2',),
    'signal_species': ('RNA_0',),
    'filenames_train_config': (f'{data_dir}/raw/summarise_simulation/2024_12_05_210221/ensemble_config.json',),
    'filenames_train_table': (f'{data_dir}/raw/summarise_simulation/2024_12_05_210221/tabulated_mutation_info.csv',),
    'filenames_verify_config': (f'{data_dir}/raw/summarise_simulation/2024_11_21_160955/ensemble_config.json',),
    'filenames_verify_table': (f'{data_dir}/raw/summarise_simulation/2024_11_21_160955/tabulated_mutation_info.csv',),
    'use_test_data': False,
    # 'total_ds': None,   # TO BE RECORDED
    'total_ds_max': 5e6,
    'train_split': 0.8,
    'x_type': 'energies',
    # XY filtering:
    'filt_x_nans': True,
    'filt_y_nans': True,
    'filt_sensitivity_nans': True,
    'filt_precision_nans': True,
    'filt_n_same_x_max': 1,
    'filt_n_same_x_max_bins': 15,
    # XY preprocessing:
    'prep_x_standardise': False,
    'prep_y_standardise': False,
    'prep_x_min_max': False,
    'prep_y_min_max': False,
    'prep_x_robust_scaling': True,
    'prep_y_robust_scaling': True,
    'prep_x_logscale': False,
    'prep_y_logscale': False,
    'prep_x_categorical': False,
    'prep_y_categorical': True,
    'prep_x_categorical_onehot': False,
    'prep_y_categorical_onehot': True,
    'prep_x_categorical_n_bins': 10,
    'prep_y_categorical_n_bins': 10,
    'prep_x_categorical_method': 'quantile',
    'prep_y_categorical_method': 'quantile',
    'prep_x_negative': True,
    'prep_y_negative': False
}

hpos_biological = {
    'n_species': 3,
    'sequence_length': 20,
    'signal_function': 'step_function',
    'signal_target': 2,
    'starting_copynumbers_input': 200,
    'starting_copynumbers_output': 200,
    'starting_copynumbers_other': 200,
    'association_binding_rate': 1000000,
    'include_prod_deg': False,
}

hpos_eval = {
    'eval_n_to_sample': 1e5
}

info_to_be_recorded = {
    'filename_saved_model': 'TO_BE_RECORDED',
    'total_ds': 'TO_BE_RECORDED',
    'n_batches': 'TO_BE_RECORDED',
    'R2_train': 'TO_BE_RECORDED',
    'R2_test': 'TO_BE_RECORDED',
    'mutual_information_conditionality': 'TO_BE_RECORDED',
    'n_layers_enc': 'TO_BE_RECORDED',
    'n_layers_dec': 'TO_BE_RECORDED',
    'run_successful': 'TO_BE_RECORDED',
    'error_msg': 'TO_BE_RECORDED',
}

hpos_all = {}
for d in [hpos_architecture, hpos_training, hpos_optimization, hpos_dataset, hpos_eval, info_to_be_recorded]:
    hpos_all.update(d)



In [60]:
df_hpos = pd.DataFrame.from_dict(hpos_all, orient='index').T
assert df_hpos.columns.duplicated().sum() == 0, 'Change some column names, there are duplicates'
basic_setting = df_hpos.copy()
df_hpos

,seed_arch,hidden_size,enc_ls,dec_ls,num_enc_layers,num_dec_layers,factor_expanding_ls,factor_contracting_ls,model,use_sigmoid_decoder,...,filename_saved_model,total_ds,n_batches,R2_train,R2_test,mutual_information_conditionality,n_layers_enc,n_layers_dec,run_successful,error_msg
0,1,16,32,32,2,2,1,1,CVAE,False,...,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED,TO_BE_RECORDED


In [61]:
# for k, v in hpos_all.items():
#     if type(v) == tuple:
#         print(k, v)
#         df_hpos[k] = df_hpos[k].apply(lambda x: tuple(x))
df_hpos['filenames_train_config']        

0    (../data/raw/summarise_simulation/2024_12_05_2...
Name: filenames_train_config, dtype: object

### All parameters

In [62]:
hpos_to_vary_from_og = {
    'seed_arch': [1, 2, 3, 4, 5],
    'num_enc_layers': [4, 5],
}
hpos_to_vary_together = {
    'hidden_size': [1, 2, 4, 8, 16, 32, 64, 128],
    'enc_ls': [4, 8, 16, 32, 64],
    'num_enc_layers': [1, 2, 3],
    'factor_expanding_ls': [1, 2, 3],
}
hpos_to_vary_together2 = {
    'hidden_size': [16, 32, 64],
    'objective_col': ['adaptability', 'sensitivity_wrt_species-6'],
    'x_type': ['energies', 'binding_rates_dissociation'],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'use_l2_reg': [True],
    'l2_reg_alpha': [0, 1e-2, 1e-3, 1e-4],
    'kl_weight': [1e-4, 1e-3, 1e-2, 1e-1, 1],
}

df_hpos.loc[df_hpos['objective_col'] == 'sensitivity_wrt_species-6', 'prep_y_logscale'] = True

In [63]:
def keep_equal(df):
    pairs = {
        'enc_ls': 'dec_ls',
        'num_enc_layers': 'num_dec_layers',
        'factor_expanding_ls': 'factor_contracting_ls',
    }
    for k1, k2 in pairs.items():
        if k1 in df.columns and k2 in df.columns:
            df[k2] = df[k1]
    return df


def add_combinatorial_keys(df_hpos, hpos_to_vary_together, basic_setting):
    keys_vary_together = sorted(hpos_to_vary_together.keys())
    for v in itertools.product(*[hpos_to_vary_together[h] for h in keys_vary_together]):
        curr = basic_setting.assign(
            **{h: vv for h, vv in zip(keys_vary_together, v)})
        df_hpos = pd.concat([df_hpos, curr], ignore_index=True)
    return df_hpos


def add_single_hpos(df_hpos, hpos_to_vary_from_og, basic_setting):
    for h, v in hpos_to_vary_from_og.items():
        df_hpos = pd.concat(
            [df_hpos] + [basic_setting.assign(**{h: vv}) for vv in v], ignore_index=True)
    return df_hpos


def postproc(df_hpos):
    df_hpos = keep_equal(df_hpos)
    df_hpos.loc[df_hpos['x_type'] ==
                'binding_rates_dissociation', 'prep_x_negative'] = False
    df_hpos = df_hpos.drop_duplicates().reset_index(drop=True)
    return df_hpos


df_hpos = add_combinatorial_keys(df_hpos, hpos_to_vary_together, basic_setting)
df_hpos = add_combinatorial_keys(
    df_hpos, hpos_to_vary_together2, basic_setting)
df_hpos = add_single_hpos(df_hpos, hpos_to_vary_from_og, basic_setting)
df_hpos = postproc(df_hpos)

# Reorder columns
cols_priority = list(set(list(hpos_to_vary_from_og.keys(
)) + list(hpos_to_vary_together.keys()) + list(hpos_to_vary_together2.keys())))
df_hpos = df_hpos[cols_priority +
                  [c for c in df_hpos.columns if c not in cols_priority]]

df_hpos.reset_index(drop=True)
len(df_hpos)

1086

# Use table to create dataset for training

In [64]:
# fn = '../data/raw/summarise_simulation/2024_11_21_144918/tabulated_mutation_info.csv'
# # fn = '../data/raw/summarise_simulation/2024_11_21_160955/tabulated_mutation_info.csv'
# # fn = '../data/raw/summarise_simulation/2024_12_05_210221/tabulated_mutation_info.csv'
# data = pd.read_csv(fn).iloc[:100]
# len(data)

In [65]:
# hpos = df_hpos.reset_index().iloc[0]
# cvae_scan(hpos, top_dir=os.path.join('data', make_datetime_str()))


In [66]:
# REMINDER: Total ds is 3 and the test data is not being used
# cvae_scan(hpos, top_dir=os.path.join('data', make_datetime_str()))

top_dir = os.path.join('data', make_datetime_str())
for i in range(len(df_hpos)):
    hpos = df_hpos.reset_index().iloc[i]
    hpos = cvae_scan(hpos, top_dir=top_dir)
    # try:
    #     try:
    #         hpos = cvae_scan(hpos, top_dir=top_dir)
    #         hpos.loc['run_successful'] = True
    #         hpos.loc['error_msg'] = ''
    #     except Exception as e:
    #         print(e)
    #         hpos.loc['run_successful'] = False
    #         hpos.loc['error_msg'] = str(e)
    # except:
    #     hpos.loc['run_successful'] = False
        
    df_hpos.loc[i] = pd.Series(hpos) if type(hpos) == dict else hpos.drop('index')
    # df_hpos.loc[i] = pd.DataFrame.from_dict(hpos).drop('index')
    if not os.path.exists(top_dir):
        os.makedirs(top_dir)
    df_hpos.to_csv(os.path.join(top_dir, 'df_hpos.csv'))
    break

train.py:train():121: Epoch 0 / 2000 -		 Train loss: 0.5934411287307739	Val loss: 0.5729807615280151	Val accuracy: 0.10586357861757278 INFO
train.py:train():121: Epoch 10 / 2000 -		 Train loss: 0.09687136858701706	Val loss: 0.09640365839004517	Val accuracy: 0.3506804406642914 INFO
train.py:train():121: Epoch 20 / 2000 -		 Train loss: 0.09033536165952682	Val loss: 0.09130507707595825	Val accuracy: 0.36210939288139343 INFO
train.py:train():121: Epoch 30 / 2000 -		 Train loss: 0.08646473288536072	Val loss: 0.085908442735672	Val accuracy: 0.48531585931777954 INFO
train.py:train():121: Epoch 40 / 2000 -		 Train loss: 0.08513110131025314	Val loss: 0.08473759889602661	Val accuracy: 0.5136215090751648 INFO
train.py:train():121: Epoch 50 / 2000 -		 Train loss: 0.08426883071660995	Val loss: 0.08396139740943909	Val accuracy: 0.524903416633606 INFO


KeyboardInterrupt: 